<a href="https://colab.research.google.com/github/poovendhiranr/Test_public_spark/blob/main/Test_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
import pyspark.sql.functions as f
from pyspark.sql import Window
spark = SparkSession.builder.master("local[*]") \
                    .appName('Test_spark') \
                    .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 48.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=74dfcd4ec832e39bd12fb0fc11d6cd655bcdd73eb8be77efc675bdf55730ccdd
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
rdd1=spark.sparkContext.textFile('/content/property.csv')
header=rdd1.map(lambda x:x.split("|")).first()
cus_header=spark.sparkContext.parallelize([['popid','size','final']])
rdd2=rdd1.map(lambda x:x.split("|")).filter(lambda x:x!=header).map(lambda x:[ x[0],x[5],str(int(x[5]) * int(x[6])) ])
rdd3=cus_header.union(rdd2)
for i in rdd3.collectAsMap():
  print(i)




ValueError: ignored

In [ ]:
df1=spark.range(1,500000).toDF("id1").withColumn("C1",f.lit("hello"))
df2=spark.range(200,600000).toDF("id2").withColumn("C2",f.lit("hi"))
df3=df1.join(df2,df1.id1==df2.id2,how="inner")
print(df3.rdd.getNumPartitions())
df3.coals(300).write.mode("overwrite").save("/content/check.csv")

100


In [ ]:
# df1 = spark.read.format("csv") \
#             .option("delimiter", "|") \
#             .option("header", "true") \
#             .option("inferSchema", "True") \
#             .load("/content/oth.csv").createOrReplaceTempView("hello")

# df2=df1.drop("_c0")
# df3=df2.groupBy("id","name").agg(f.min(df2.age).alias ("age") ,f.min(df2.city))
# df3.show()

# spark.sql('''select id, name,min(age) as age ,min(city) as city from hello group by id,name''').show()
# df1 = spark.read.format("csv") \
#             .option("delimiter", "|") \
#             .option("header", "true") \
#             .option("inferSchema", "True") \
#             .load("/content/oth1.csv")
         
#spark.sql('''select Name,Weight from (select Name,Weight,dense_rank() over(order by Weight desc) as rnk from hello )temp where rnk<3 ''').show()



# wd=Window.orderBy(f.col("Weight").desc())
# df1.withColumn("rnk",f.rank().over(wd)).filter(f.col("rnk")<3).select(f.col("Name"),f.col("Weight")).show()

# rdd1=spark.sparkContext.textFile("/content/oth1.csv")
# header = rdd1.map(lambda x:x.split('|')).first()
# ab=rdd1.map(lambda x:x.split('|')).filter(lambda x: x!=header) \
#        .sortBy(lambda X:X[1],False) \
#        .filter(lambda x : int(x[1])>60)
# for a in ab.collect():
#   print(a)


# ab=rdd1.map(lambda x:x.split('|')).filter(lambda x: x!=header) \
#        .sortBy(lambda X:X[1],False) \
#        .take(2)
# print(ab[1])


# df1 = spark.read.format("csv") \
#             .option("delimiter", ",") \
#             .option("header", "true") \
#             .option("inferSchema", "True") \
#             .load("/content/oth2.csv")
# df1.withColumn("hob",f.explode(f.split(f.col("hobbies"),'-'))) \
#    .drop(f.col("hobbies")).show()

# rdd1=spark.sparkContext.textFile("/content/oth2.csv")
# header = rdd1.map(lambda x:x.split(',')).first()
# ab=rdd1.map(lambda x:x.split(',')).filter(lambda x: x!=header).map(lambda x :[x[0],x[1],x[2].split("-")]) 
# lis=[]
# for a in ab.collect():
#   for b in a[2]:
#     lis=lis+[a[0],a[1],b]
# print(lis)    



# data = [("James","Sales",34), ("Michael","Sales",56),("Robert","Sales",30), ("Maria","Finance",24) ]
# df1 = data.toDF("name","dept","age")
# df1.printSchema()

# data2=Seq(("James","Sales","NY",9000),("Maria","Finance","CA",9000),("Jen","Finance","NY",7900),("Jeff","Marketing","CA",8000))
# df2 = data2.toDF("name","dept","state","salary")
# df2.printSchema()


# lis=[1,0,1,0,1,0,1,0]
# ad=[]
# for i in lis: 
#   if i<1:
#     ad=ad+[i]
# for i in lis:
#   if i>0:
#     ad=ad+[i]
# print(ad)       





[0, 0, 0, 0, 1, 1, 1, 1]


SESSION_Allocation

In [15]:

df1 = spark.read.format("csv") \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("TimeStampFormat", "yyyy-MM-dd HH:mm") \
            .option("inferSchema", "True") \
            .load("/content/session2.csv")

# populate prev dtime for each record
df_30 = df1.withColumn("prev_dtime",f.lag("dtime").over(Window.partitionBy("idt").orderBy("dtime")),)

#find the diff of dtime and prev_dtime if it is less than or equal to 30 min add 0 else add 1
df_30 = df_30.withColumn("session_diff_30min", \
                   (f.col("dtime").cast(LongType()) - f.col("prev_dtime").cast(LongType()))/60)
df_30 = df_30.withColumn("session_flag_30min", \
                  f.when(f.col("session_diff_30min")<30,0).otherwise(1))

df_30 = df_30.withColumn("session_id_30min",f.sum("session_flag_30min").over(Window.partitionBy("idt").orderBy("dtime")))

#df_30_mindt = df_30.groupBy("idt", "session_id_30min").agg(f.min(f.col("dtime")).alias("min_dtime"))
df_30_mindt = df_30.groupBy("idt").agg(f.min(f.col("dtime")).alias("min_dtime"))

df_2hrs_1 = df_30.join(df_30_mindt, ["idt" ], "inner")
df_2hrs_1=df_2hrs_1.withColumn("session_id_2hrs", \
                   f.when(f.floor((f.col("dtime").cast(LongType()) - f.col("min_dtime").cast(LongType()))/7200)>=1,1).otherwise(0))

df_2hrs_1=df_2hrs_1.withColumn("total",f.col("session_id_2hrs")+f.col("session_id_30min"))







# create the session id using the running total
# df_30 = df_30.withColumn("session_id_30min",f.sum("session_flag_30min").over(Window.partitionBy("idt").orderBy("dtime")))
# df_30.show(truncate=False)

#now we have completed the creating the session id with 30 min logic
#start implement the 2hrs logic
#for each user and session id get the minimum date time
# df_30_mindt = df_30.groupBy("idt", "session_id_30min").agg(f.min(f.col("dtime")).alias("min_dtime"))
# df_30_mindt.show(truncate=False)

#now join df_30 with df_30mindt to populate the min_dtime column
# df_2hrs_1 = df_30.join(df_30_mindt, ["idt" , "session_id_30min"], "inner")

# #now find the differeent between dtime and min_dtime if the difference is less than 7200 (2HRS) add 0 else 1
# df_2hrs_2 = df_2hrs_1.withColumn("session_flag_2hrs", \
#         f.when(f.col("dtime").cast(LongType()) - f.col("min_dtime").cast(LongType()) <= 7200, 0).otherwise(1))


# df_2hrs_2 = df_2hrs_2.withColumn("session_flag_2hrs",f.sum("session_flag_2hrs").over(Window.partitionBy("idt").orderBy("dtime")))


# #now to populate the final session id sum session_flag_30min and session_flag_2hrs
#df_final = df_2hrs_1.withColumn("total", f.col("session_id_30min") + f.col("session_id_2hrs"))

df_2hrs_1.show(truncate=False)





+---+-------------------+-------------------+------------------+------------------+----------------+-------------------+---------------+-----+
|idt|dtime              |prev_dtime         |session_diff_30min|session_flag_30min|session_id_30min|min_dtime          |session_id_2hrs|total|
+---+-------------------+-------------------+------------------+------------------+----------------+-------------------+---------------+-----+
|U1 |2020-03-28 03:45:00|null               |null              |1                 |1               |2020-03-28 03:45:00|0              |1    |
|U1 |2020-03-28 04:10:00|2020-03-28 03:45:00|25.0              |0                 |1               |2020-03-28 03:45:00|0              |1    |
|U1 |2020-03-28 04:35:00|2020-03-28 04:10:00|25.0              |0                 |1               |2020-03-28 03:45:00|0              |1    |
|U1 |2020-03-28 04:55:00|2020-03-28 04:35:00|20.0              |0                 |1               |2020-03-28 03:45:00|0              |1    |

In [ ]:
df1 = spark.read.format("csv") \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("TimeStampFormat", "yyyy-MM-dd HH:mm") \
            .option("inferSchema", "True") \
            .load("/content/session2.csv")













AttributeError: ignored

In [ ]:
df=spark.read.format("csv").option("delimiter", ",").option("header",True).load("/content/fuel.csv")
df1=df.withColumn('total_min',F.hour(F.col("timestamp"))*60+F.minute(F.col("timestamp"))).createOrReplaceTempView("temp1")
df2=spark.sql('''
select trip_id,timestamp,fuel_level,LEAD(fuel_level ,1,0) 
OVER ( ORDER BY timestamp,0) AS compare_fuel,
(fuel_level-LEAD(fuel_level ,1,0) 
OVER ( ORDER BY timestamp,0)) as Diff
from temp1''').createOrReplaceTempView("temp2")
df3=spark.sql('''select avg(Diff),trip_id  from temp2 group by trip_id ''').show()

+-----------------+--------+
|        avg(Diff)| trip_id|
+-----------------+--------+
|7.888888888888889|trip_006|
+-----------------+--------+



In [ ]:
df_1=spark.read.format("csv").option("header",True).load("/content/check_22.csv")
df=df_1.createOrReplaceTempView("check7")
fg=spark.sql('''select idf,sum(amount),count(idf) as occurance from check7 where idf in 
(select i.idf from check7 as i group by idf having count( distinct orderdate) >1)
 group by idf ''').createOrReplaceTempView("check77")
gg=spark.sql('''select * from check77''').show()




+---+-----------+---------+
|idf|sum(amount)|occurance|
+---+-----------+---------+
|101|      600.0|        2|
+---+-----------+---------+



In [ ]:
df=spark.createDataFrame([('translate',)], ['a']).select(F.translate('a', "rnlt", "123").alias('r')).collect()
print(df)

[Row(r='1a2s3ae')]


In [ ]:
dF = spark.createDataFrame
([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
df.select(F.explode(df.intlist).alias("anInt")).collect()

NameError: ignored

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .appName('Test_spark') \
                    .getOrCreate()

KeyboardInterrupt: ignored

In [ ]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       print (x[0:1].upper())
       print( x[1:len(x)])
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
       
    return resStr

convertCase("hello world")


H
ello
W
orld


'Hello World '

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

""" Converting function to UDF """
convertUDF = udf(lambda z: convertCase(z))

df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
.show(truncate=False)

def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
.show(truncate=False)

""" Using UDF on SQL """
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + \
          "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  
     
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  

In [ ]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)


+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+



In [ ]:
df2.select([F.regexp_replace(F.col(cname),' ','').alias (F.regexp_replace(cname,' ','')) for cname in df2.columns]).show()

TypeError: ignored

In [ ]:
# prime number testing
Last_number=12
sum=0

for number in range(2, Last_number + 1):
    print("next",number)
    
    for i in range(2, number):
        
        if (int(number % i) == 0):
           
            i = number
            print("check",i)
            break;

    print("integervalue",i)
    print("numbervalue",number)
    if i is not number:
      print("valuexxx",number)
      sum += number
print("\nThe sum of prime numbers in python from 1 to ", Last_number, " is :", sum)


In [ ]:
a= spark.sparkContext.parallelize(Array(("a",1),("a",2),("b",2)))
b =a.foldByKey(1)(_+_)

NameError: ignored

In [ ]:
import  pyspark.sql.functions as  F
df=spark.createDataFrame([('translate',)], ['a']).select(F.translate('a', "rnlt", "123").alias('r')).collect()
print(df)



[Row(r='1a2s3ae')]


In [ ]:
df = spark.createDataFrame([("a", 1), ("b", 2), ("c",  3)], ["Col1", "Col2"])
df.show()
df.select(df.colRegex("`(Col1)?+.+`")).show()

+----+----+
|Col1|Col2|
+----+----+
|   a|   1|
|   b|   2|
|   c|   3|
+----+----+

